In [ ]:
pip install -r requirements.txt

In [1]:
from langchain_openai import AzureChatOpenAI
from langchain.tools import tool
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent
from langgraph.graph import StateGraph, START, END, Graph 
from models.database import *
from models.models import *
import langgraph
import os
from sqlalchemy.orm import Session
import json
#from dotenv import load_dotenv

In [2]:
#load_dotenv()
MODELO = os.getenv("MODELO")
KEY = os.getenv("KEY")
VERSION = os.getenv("VERSION")
ENDPOINT = os.getenv("ENDPOINT")

In [3]:
llm = AzureChatOpenAI(
    api_key = KEY,
    api_version = VERSION,
    azure_endpoint = ENDPOINT,
    azure_deployment = MODELO
    )

In [ ]:
import openai
openai_client = openai.AzureOpenAI(
    api_key = KEY,
    api_version = VERSION,
    azure_endpoint = ENDPOINT
    )

In [ ]:
messages=[{"role": "system", "content": "Eres un asistente útil."},
        {"role": "user", "content": "¿Cuál es la capital de España?"}]

response = openai_client.chat.completions.create(
    model = MODELO,
    messages = messages,
    max_tokens=100
    )
response.choices[0].message.content

    ("pendiente_condicionada", "flujo"),
    ("pendiente_no_condicionada", "flujo"),
    ("grabar_cesta_srm", "flujo"),
    ("solicitar_firma_inf_prov_cond", "flujo"),
    ("firma_inf_prov_cond", "flujo"),
    ("lanzar_cesta_srm", "flujo"),
    ("aprobar_compra_manager", "aprobacion"),
    ("aprobar_compra_gestion", "aprobacion"),
    ("revision_compra", "flujo"),
    ("solicitar_oferta_provedoores", "flujo"),
    ("proveedor_envia_ofertn", "flujo"),
    ("consulta_acep_oferta", "flujo"),
    ("negociacion_proveedores", "flujo"),
    ("proponer_proveedor", "flujo"),
    ("aprobar_adjuducacion", "aprobacion"),
    ("facturar", "flujo"),
    ("fin", "flujo"),

In [ ]:
# Agrego un proveedor de prueba

def agregar_proveedor_prueba():
    """
    Agrega un proveedor de prueba a la base de datos.
    """
    session = Session(bind=engine)

    # Crear un proveedor de prueba
    nuevo_proveedor = Proveedor(
        nombre="Apple",
        contacto="Soporte Apple",
        email="apple@proveedor.com",
        estado="aprobado"
    )
    session.add(nuevo_proveedor)
    session.commit()

    print(f"✅ Proveedor de prueba añadido: {nuevo_proveedor.nombre} (ID: {nuevo_proveedor.id_proveedor})")
    session.close()

# Ejecutar la función para agregar el proveedor
agregar_proveedor_prueba()


PENDIENTE QUE SE GENERE UNA DESCRIPCION AUTOMATICA POR LA IA Y LE PONGA NOMBRE A LA CESTA EL AGENTE OBTENER DATOS PEDIDO.

In [4]:
def obtener_datos_pedido(entrada_usuario):
    ''' Nodo que usa IA para obtener los datos del pedido del usuario, crea la cesta y el pedido con el estado inicial,
      y decide si el flujo valida el proveedor o graba la cesta.
    '''
    # Se crea la sesión de SQLAlchemy	
    session = Session(bind=engine)

    prompt = f''' 
    Eres el encargado de obtener la información de un pedido.
    Se necesita conocer:
    - Nombre del proveedor (si no se menciona, deja "desconocido")
    - Moneda en la que se realizó la compra utiliza el codigo de 3 letras
    - Presupuesto estimado
    - Producto a comprar

    {entrada_usuario}

    Devuelve los datos en un formato claro como:
    "Proveedor: Proveedor X, Moneda: EUR, Presupuesto: 5000, Producto: [Producto Y]"
    '''
    response = llm.invoke(prompt)
    datos_ia = response.content.strip()

    print(f'IA: {datos_ia}')

    # Extraer los datos del texto
    datos_extraidos = {}
    for item in datos_ia.split(','):
        key, value = item.strip().split(':')
        datos_extraidos[key.lower().strip()] = value.strip()
    
    nombre_proveedor = datos_extraidos.get('proveedor', 'desconocido')
    codigo_moneda = datos_extraidos.get('moneda', 'EUR') # Por defecto EUR
    presupuesto = float(datos_extraidos.get('presupuesto', 1000)) # Por defecto 1000

    print(f"📦 Datos extraídos: Proveedor: {nombre_proveedor}, Moneda: {codigo_moneda}, Presupuesto: {presupuesto}")
    
    # Verificar si el proveedor ya existe en la BD
    proveedor = session.query(Proveedor).filter_by(nombre=nombre_proveedor).first()
    proveedor_id = proveedor.id_proveedor if proveedor else None

    if proveedor:
        print(f"✅ Proveedor '{proveedor.nombre}' encontrado en la base de datos.")
        es_nuevo = False
        estado_inicial_nombre = "pendiente_no_condicionada"
    else:
        print(f"⚠️ Proveedor '{nombre_proveedor}' no encontrado. Requiere validación.")
        es_nuevo = True
        estado_inicial_nombre = "pendiente_condicionada"
    
    # Obtener estado inicial correspondiente
    estado_inicial = session.query(Estado).filter_by(nombre=estado_inicial_nombre).first()
    if not estado_inicial:
        print(f"⚠️ Estado '{estado_inicial_nombre}' no encontrado en la base de datos.")
        return {"error": "Estado inicial no encontrado"}



    # Verificar que la moneda existe en la BD
    moneda = session.query(Moneda).filter(Moneda.codigo == codigo_moneda).first()
    moneda_id = moneda.id_moneda if moneda else None

    if not moneda:
        print(f"⚠️ Moneda '{codigo_moneda}' no encontrada. Se usará EUR por defecto.")
        moneda = session.query(Moneda).filter_by(codigo="EUR").first()
        moneda_id = moneda.id_moneda

    # Crear la cesta y el pedido
    nueva_cesta = Cesta(
        nombre="Cesta generada automáticamente",
        descripcion="Pedido generado automáticamente",
        tipo_compra="ordinaria",
        presupuesto=presupuesto,
        usuario_sap_id=1,
        proveedor_id=proveedor_id,
        moneda_id=moneda_id,
        contrato_id=None
    )
    session.add(nueva_cesta)
    session.commit()

    print(f"✅ Cesta creada con ID: {nueva_cesta.id_cesta}")

    nuevo_pedido = Pedido(
        posicion=1,
        tipo="Ordinario",
        pedido_tipoimp="K",
        descripcion=f"Compra de {datos_extraidos.get('productos', 'productos no especificados')}",
        id_proveedor=proveedor_id,
        cesta_id=nueva_cesta.id_cesta,  # La Cesta ya está asignada
        estado_tramitacion_id=estado_inicial.id_estado,
        creador_id=1,
        moneda_id=moneda_id
    )
    session.add(nuevo_pedido)
    session.commit()

    # Se extraen las variables antes de cerrar la sesion 

    pedido_id = nuevo_pedido.id_pedido
    cesta_id = nueva_cesta.id_cesta
    session.close()

    return {

        "id_pedido": pedido_id,
        "id_proveedor": proveedor_id,  # ✅ Ya no intenta acceder a un objeto desvinculado
        "nombre_proveedor": nombre_proveedor,
        "id_cesta": cesta_id,
        "id_moneda": moneda_id,  # ✅ Ya no intenta acceder a un objeto desvinculado
        "estado_actual": estado_inicial_nombre,
        "id_usuario": 1,
        "nombre_cesta": "Cesta generada automáticamente",
        "tipo_compra": "ordinaria",
        "presupuesto": presupuesto,
        "descripcion_pedido": "Pedido generado automáticamente",
        "pedido_tipoimp": "K",
        "es_nuevo": es_nuevo  # 🔹 Indica si el proveedor debe validarse antes de continuar
    }


In [10]:
consulta_usuario = """
Necesito comprar 15 monitores y 10 teclados mecánicos para la oficina. 
El presupuesto estimado es de 7000 dólares. 
Quiero hacer la compra a Apple, pero si no es posible, podemos buscar otro proveedor.
"""

# Probar obtener_datos_pedido con una consulta de usuario
resultado = obtener_datos_pedido(consulta_usuario)

# Mostrar los resultados obtenidos
print("\n🔹 RESULTADO DE LA FUNCIÓN 🔹")
for clave, valor in resultado.items():
    print(f"{clave}: {valor}")

# Verificar si el proveedor fue detectado correctamente
if resultado["es_nuevo"]:
    print("\n⚠️ El proveedor es nuevo y requiere validación.")
else:
    print("\n✅ El proveedor ya existe en la base de datos y se usará directamente.")


IA: Proveedor: Apple, Moneda: USD, Presupuesto: 7000, Producto: Monitores y Teclados Mecánicos
📦 Datos extraídos: Proveedor: Apple, Moneda: USD, Presupuesto: 7000.0
2025-03-18 18:19:27,924 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-18 18:19:27,928 INFO sqlalchemy.engine.Engine SELECT proveedores.id_proveedor AS proveedores_id_proveedor, proveedores.nombre AS proveedores_nombre, proveedores.contacto AS proveedores_contacto, proveedores.email AS proveedores_email, proveedores.estado AS proveedores_estado, proveedores.fecha_registro AS proveedores_fecha_registro 
FROM proveedores 
WHERE proveedores.nombre = ?
 LIMIT ? OFFSET ?
2025-03-18 18:19:27,928 INFO sqlalchemy.engine.Engine [generated in 0.00089s] ('Apple', 1, 0)
✅ Proveedor 'Apple' encontrado en la base de datos.
2025-03-18 18:19:27,932 INFO sqlalchemy.engine.Engine SELECT estados.id_estado AS estados_id_estado, estados.nombre AS estados_nombre, estados.tipo AS estados_tipo, estados.requiere_aprobacion AS estados_requie

In [12]:
resultado

{'id_pedido': 5,
 'id_proveedor': 1,
 'nombre_proveedor': 'Apple',
 'id_cesta': 5,
 'id_moneda': 3,
 'estado_actual': 'pendiente_no_condicionada',
 'id_usuario': 1,
 'nombre_cesta': 'Cesta generada automáticamente',
 'tipo_compra': 'ordinaria',
 'presupuesto': 7000.0,
 'descripcion_pedido': 'Pedido generado automáticamente',
 'pedido_tipoimp': 'K',
 'es_nuevo': False}

In [5]:
def decidir_siguiente_nodo(datos):
    """
    Determina si el flujo sigue a `grabar_cesta_srm` o `gestionar_proveedor_nuevo`
    dependiendo de si el proveedor es nuevo o no.
    """
    if datos["es_nuevo"]:
        return "gestionar_proveedor_nuevo", datos
    return "grabar_cesta_srm", datos


In [6]:
def decidir_siguiente_nodo(datos):
    """
    Determina si el flujo sigue a `grabar_cesta_srm` o `gestionar_proveedor_nuevo`
    dependiendo de si el proveedor es nuevo o no.
    """

    # 📌 Debugging: Imprimir el tipo de datos para asegurarnos de que es un diccionario
    print(f"📌 Tipo de datos recibido en decidir_siguiente_nodo: {type(datos)}")

    # 📌 Si `datos` es una tupla, tomar el segundo elemento
    if isinstance(datos, tuple):
        datos = datos[1]  # 🔹 Solo nos interesa la segunda parte de la tupla

    # 📌 Validar si el proveedor es nuevo
    if datos.get("es_nuevo", False):
        print("🔄 Redirigiendo a `gestionar_proveedor_nuevo`...")
        return "gestionar_proveedor_nuevo", datos
    else:
        print("✅ Redirigiendo a `grabar_cesta_srm`...")
        return "grabar_cesta_srm", datos


In [ ]:
def grabar_cesta_srm():
    tools = []
    agent_pedido = create_react_agent(llm, tools)


In [7]:
def grabar_cesta_srm(datos):
    """
     Simula el envío del pedido a SAP SRM, registra el estado en la BD y actualiza `HistorialPedido`
    antes de cambiar el estado a `aprobar_compra_manager`.
    """
    session = Session(bind=engine)

    if isinstance(datos, tuple):
        datos = datos[1]


    id_pedido = datos["id_pedido"]

    #  Obtener el Pedido de la base de datos
    pedido = session.query(Pedido).filter_by(id_pedido=id_pedido).first()
    
    if not pedido:
        print(f"❌ Error: No se encontró el Pedido {id_pedido}.")
        return {"error": "Pedido no encontrado"}

    #  Guardar el estado inicial del Pedido antes de enviarlo a SAP
    estado_anterior_id = pedido.estado_tramitacion_id

    #  Crear JSON para enviar a SAP SRM
    data_sap = {
        "pedido_id": pedido.id_pedido,
        "descripcion": pedido.descripcion,
        "tipo": pedido.tipo,
        "proveedor_id": pedido.id_proveedor,
        "moneda_id": pedido.moneda_id,
        "cesta_id": pedido.cesta_id
    }

    json_sap = json.dumps(data_sap, indent=4)  # 🔹 Simulación de envío
    print(f"📤 Simulando envío de Pedido {id_pedido} a SAP SRM en formato JSON:\n{json_sap}")

    response_data = {
        "status": "success",
        "message": "Pedido registrado correctamente en SAP SRM"
    }

    print(f"✅ Pedido {id_pedido} registrado en SAP SRM (simulación).")
    

    # Se cambia el estado de la tabla pedido a aprobar compra manager
    nuevo_estado = session.query(Estado).filter_by(nombre="grabar_cesta_srm").first()
    if not nuevo_estado:
        print("❌ Error: No se encontró el estado 'grabar_cesta_srm'.")
        return {"error": "Estado no encontrado"}
    
    # Se obtiene la transicion correspondiente en la tabla transiciones
    transicion = session.query(Transicion).filter_by(
        estado_origen_id=estado_anterior_id,
        estado_destino_id=nuevo_estado.id_estado
    ).first()

    if not transicion:
        print(f"⚠️ Advertencia: No se encontró una transición desde {estado_anterior_id} a {nuevo_estado.id_estado}.")
        return {"error": "Transición no encontrada"}

    # Se registra el cambio en el historial del pedido
    historial = HistorialPedido(
        id_pedido=pedido.id_pedido,
        estado_anterior=estado_anterior_id,  # 🔹 Estado antes de SAP
        estado_nuevo=nuevo_estado.id_estado,  # 🔹 Estado después de SAP
        estado_aprobacion="pendiente",
        id_usuario=pedido.creador_id,
        id_transicion=transicion.id_transicion
    )
    session.add(historial)
    session.commit()

    print(f" Cambio registrado en HistorialPedido para Pedido {id_pedido}.")
    
    #  Ahora cambiar el estado del Pedido
    pedido.estado_tramitacion_id = nuevo_estado.id_estado
    session.commit()

    print(f"🔄 Pedido {id_pedido} actualizado al estado '{nuevo_estado.nombre}'.")

    session.close()

    return {
        "id_pedido": id_pedido,
        "estado_actual": nuevo_estado.nombre,
    }

In [8]:
def aprobacion_proveedor():
    tools = []
    agent_pedido = create_react_agent(llm, tools)
    print("PENDIENTE APROBACION PROVEEDOR")

def aprobar_compra_manager():
    tools = []
    agent_pedido = create_react_agent(llm, tools)
    print("PENDIENTE APROBACION PROVEEDOR")

def gestionar_proveedor_nuevo():
    tools = []
    agent_pedido = create_react_agent(llm, tools)
    print("PENDIENTE GESTIONAR PROVEEDOR NUEVO")

In [23]:
datos = grabar_cesta_srm(resultado)

2025-03-19 17:57:14,839 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-19 17:57:14,840 INFO sqlalchemy.engine.Engine SELECT pedidos.id_pedido AS pedidos_id_pedido, pedidos.posicion AS pedidos_posicion, pedidos.tipo AS pedidos_tipo, pedidos.pedido_tipoimp AS pedidos_pedido_tipoimp, pedidos.descripcion AS pedidos_descripcion, pedidos.fecha_creacion AS pedidos_fecha_creacion, pedidos.fecha_actualizacion AS pedidos_fecha_actualizacion, pedidos.estado_tramitacion_id AS pedidos_estado_tramitacion_id, pedidos.creador_id AS pedidos_creador_id, pedidos.contrato_id AS pedidos_contrato_id, pedidos.cesta_id AS pedidos_cesta_id, pedidos.moneda_id AS pedidos_moneda_id, pedidos.id_proveedor AS pedidos_id_proveedor 
FROM pedidos 
WHERE pedidos.id_pedido = ?
 LIMIT ? OFFSET ?
2025-03-19 17:57:14,841 INFO sqlalchemy.engine.Engine [cached since 519.8s ago] (5, 1, 0)
📤 Simulando envío de Pedido 5 a SAP SRM en formato JSON:
{
    "pedido_id": 5,
    "descripcion": "Compra de productos no especifica

In [9]:
# Codigo para generar el flujo

#graph_builder = StateGraph(dict)
graph_builder = Graph()

# Nodos al grafo

graph_builder.add_node("obtener_datos_pedido", obtener_datos_pedido)
graph_builder.add_node("decidir_siguiente_nodo", decidir_siguiente_nodo)
graph_builder.add_node("gestionar_proveedor_nuevo", gestionar_proveedor_nuevo)
graph_builder.add_node("grabar_cesta_srm", grabar_cesta_srm)
graph_builder.add_node("aprobar_compra_manager", aprobar_compra_manager)
graph_builder.add_node("aprobacion_proveedor", aprobacion_proveedor)


In [10]:
# Empieza en obtener datos del pedido
graph_builder.add_edge(START, "obtener_datos_pedido")

graph_builder.add_edge("obtener_datos_pedido", "decidir_siguiente_nodo")

# 📌 Configurar la decisión basada en la existencia del proveedor
graph_builder.add_conditional_edges("decidir_siguiente_nodo", decidir_siguiente_nodo)

# 📌 Conectar `gestionar_proveedor_nuevo` con `grabar_cesta_srm` para continuar el flujo
graph_builder.add_edge("gestionar_proveedor_nuevo", "grabar_cesta_srm")

# Termina en grabar cesta srm
graph_builder.add_edge("grabar_cesta_srm", END)

In [ ]:
from langchain_core.tools import tool

In [11]:
graph = graph_builder.compile()

In [12]:
prompt = "Quiero comprar 20 portatiles del proveedor Apple con un presupuesto de 6600 euros"
result = graph.invoke(prompt)

IA: Proveedor: Apple, Moneda: EUR, Presupuesto: 6600, Producto: Portátiles
📦 Datos extraídos: Proveedor: Apple, Moneda: EUR, Presupuesto: 6600.0
2025-03-19 19:43:43,020 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-19 19:43:43,023 INFO sqlalchemy.engine.Engine SELECT proveedores.id_proveedor AS proveedores_id_proveedor, proveedores.nombre AS proveedores_nombre, proveedores.contacto AS proveedores_contacto, proveedores.email AS proveedores_email, proveedores.estado AS proveedores_estado, proveedores.fecha_registro AS proveedores_fecha_registro 
FROM proveedores 
WHERE proveedores.nombre = ?
 LIMIT ? OFFSET ?
2025-03-19 19:43:43,024 INFO sqlalchemy.engine.Engine [generated in 0.00096s] ('Apple', 1, 0)
✅ Proveedor 'Apple' encontrado en la base de datos.
2025-03-19 19:43:43,027 INFO sqlalchemy.engine.Engine SELECT estados.id_estado AS estados_id_estado, estados.nombre AS estados_nombre, estados.tipo AS estados_tipo, estados.requiere_aprobacion AS estados_requiere_aprobacion 
FROM 